# Fantasy Football Project

In this project we will leverage the FFP API to gain access to player and team data to enable more infomred managerial decisions. 

Ultimate end goal would be to host an app that takes in a players current team and makes suggestions for best transfers to make backed up by evidence!! We also want to incorporate a machine learning element. 


Key Link:\
https://medium.com/@frenzelts/fantasy-premier-league-api-endpoints-a-detailed-guide-acbd5598eb19#:~:text=The%20answer%20is%20simple.,before%20the%20'%2Fhistory'.

#### Notebook Aims:

In this notebook we'll create a script to execute weekly (or more regular) calls to the FPL API. We'll then construct the key datasets from which the rest of the project is developed from. For detailed descriptions on these data sets please refer to the Readme. 

In [31]:
import warnings
warnings.filterwarnings("ignore")

In [9]:
import pandas as pd
import requests
import json
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
r = requests.get(url)
json = r.json()
pd.set_option('display.max_columns', None)

In [10]:
elements_df = pd.DataFrame(json["elements"])

In [11]:
elements_df

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,id,in_dreamteam,news,news_added,now_cost,photo,points_per_game,second_name,selected_by_percent,special,squad_number,status,team,team_code,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text
0,0.0,0.0,37605,0,0,-3,3,0,3,0.0,0.0,0,Mesut,0.0,1,False,Not included in Arsenal's 25-man Premier Leagu...,2020-10-20T22:30:18.118477Z,67,37605.jpg,0.0,Özil,0.5,False,None,u,1,3,0,3441,0,54818,13,0.0,0.0,Özil,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,693,283,693,283,693,283,693,283,NaN,,NaN,,NaN,
1,0.0,0.0,39476,0,0,-2,2,0,2,0.0,0.0,0,Sokratis,0.0,2,False,Left the club by mutual consent on 20/1,2020-10-21T10:30:18.546407Z,48,39476.jpg,0.0,Papastathopoulos,0.1,False,None,u,1,3,0,10266,0,19127,3,0.0,0.0,Sokratis,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,660,234,652,235,644,232,663,234,NaN,,NaN,,NaN,
2,25.0,0.0,41270,0,0,-1,1,0,2,0.2,0.0,0,David,0.0,3,False,Knee injury - 25% chance of playing,2021-04-02T17:00:25.666921Z,54,41270.jpg,2.1,Luiz Moreira Marinho,0.8,False,None,d,1,3,40,76618,12,128590,205,0.0,7.4,David Luiz,1344,1,0,3,20,0,0,0,1,1,0,0,223,244.6,45.2,123.0,41.0,241,93,334,111,229,63,298,100,NaN,,4.0,,NaN,
3,25.0,0.0,54694,0,0,-7,7,2,3,0.4,0.0,0,Pierre-Emerick,0.5,4,False,Illness - 25% chance of playing,2021-04-12T08:30:26.681398Z,113,54694.jpg,4.3,Aubameyang,6.7,False,None,d,1,3,108,1140046,92,3815449,3588,0.0,9.6,Aubameyang,2007,9,1,7,25,1,0,0,2,0,0,10,306,428.2,292.4,731.0,144.9,128,47,109,74,33,18,49,29,NaN,,NaN,,1.0,
4,100.0,100.0,58822,0,0,-4,4,0,2,1.0,0.5,0,Cédric,0.0,5,False,,2020-09-23T09:00:14.881983Z,46,58822.jpg,2.8,Soares,0.3,False,None,a,1,3,28,40810,27,60645,379,0.0,6.1,Cédric,744,0,1,2,11,0,0,0,1,0,0,3,125,110.8,114.8,66.0,29.2,344,131,235,59,310,102,336,120,NaN,,5.0,,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688,NaN,NaN,448487,0,0,0,0,0,1,0.5,0.5,0,Andreas,0.0,617,False,,None,40,448487.jpg,0.0,Söndergaard,0.2,False,None,a,20,39,0,32248,16,17151,62,0.0,0.0,Söndergaard,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,542,62,522,36,496,31,549,61,NaN,,NaN,,NaN,
689,0.0,0.0,209353,0,0,-1,1,0,4,0.0,0.0,0,Patrick,0.0,626,False,Joined Valencia on loan until the end of the s...,2021-01-31T20:00:27.142575Z,59,209353.jpg,1.0,Cutrone,0.0,False,None,u,20,39,2,3443,0,3427,0,0.0,0.3,Cutrone,23,0,0,0,0,0,0,0,0,0,0,0,5,0.0,0.0,4.0,0.2,690,88,691,89,423,66,496,68,NaN,,NaN,,NaN,
690,NaN,NaN,465551,0,0,0,0,0,2,0.5,0.5,0,Nigel,0.0,629,False,,None,40,465551.jpg,0.0,Lonwijk,0.0,False,None,a,20,39,0,1410,4,1134,9,0.0,0.0,Lonwijk,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,546,209,526,210,500,197,553,209,NaN,,NaN,,NaN,
691,100.0,100.0,73314,0,0,-2,2,0,4,4.3,4.3,2,Willian José,3.8,642,False,,2021-01-26T14:00:25.113370Z,68,73314.jpg,2.7,Da Silva,0.5,False,None,a,20,39,35,78816,575,43655,1408,0.6,5.1,Willian José,974,1,1,5,12,0,0,0,0,0,0,3,118,82.2,152.0,217.0,44.9,371,45,205,32,157,44,278,42,NaN,,NaN,,NaN,


In [22]:
def fpl_calls():
    """
    In this function we'll call in the historical player data and store in various dataframes for future use
    """
    
    # Import requests to access website data
    import requests
    import json
    import pandas as pd
    
    # Set the endpoints
    url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
    
    
    # Call in the endpoint and store as a json file
    r = requests.get(url)
    json = r.json()
    
    ### Create the basic_player_info dataset from info available in the json
    
    # Store the latest player data into a df
    elements_df = pd.DataFrame(json["elements"])
    
    # Store selected team features info in a df 
    teams = pd.DataFrame(json["teams"])
    team = teams[["id", "name", "short_name"]]
    
    # Merge key features from each of the elements and teams dfs
    basic_player_info = elements_df[["element_type", "id", 
                            "team", "web_name", 
                            "first_name", "second_name", "code"]].merge(team, how="left", 
                                                                        left_on="team", 
                                                                        right_on="id").drop("id_y", axis=1)
    # tidy up the feature names
    basic_player_info.rename(columns={"id_x": "id",
                             "name": "team"}, inplace=True)
    
    ########################################################################################################
    
    
    ### Create the detailed season data per player set
    
    # First get the full season's data for element(player) 1
    player1 = "https://fantasy.premierleague.com/api/element-summary/1/"
    p1 = requests.get(player1)
    type(p1)
    player1_json = p1.json()
    plr1_hist = pd.DataFrame(player1_json['history'])
    
    # Now loop through all the other players data and add to player1
    players = plr1_hist
    for i in range(2, elements_df.shape[0]):
        # Access the endpoint
        player = f"https://fantasy.premierleague.com/api/element-summary/{i}/"
        # Store data as json
        get_player = requests.get(player)
        player_json = get_player.json()
        # Convert the player history in a dataframe
        player_hist = pd.DataFrame(player_json['history'])
        # Add the player history to the master player history dataset
        players = players.append(player_hist)
    
    #########################################################################################################
    
    ### Create our cumulative points dataset
    
    # First we'll select relevant features from the players dataset 
    value_set = players[["element", "fixture", "round", "value", "total_points"]]
    
    # Then select names features from the elements_df
    names = elements_df[["id", "first_name", "web_name"]]
    
    # Now merge the features
    info = value_set.merge(names, how="left", left_on="element", right_on="id").drop("id", axis=1)
    
    # Rename value header to cost to avoid future confusion with our player value metric
    info.rename(columns={"value": "cost"}, inplace=True)
    
    # Create new features to store cumulative points and player value
    info["cum_points"] = 0
    info["local_val"] = 0
    info["cum_val"] = 0
    
    # Now build the new features out 
    # Set an empty dataset to store the cumulative player value
    cumulative = pd.DataFrame(columns=info.columns)

    for j in range(1, info.element.nunique()):
        # Create a df with only the current player's history 
        current_player = info[info.element==j]

        # Loop through this history and populate the value features based on the rules we created
        for i in range(current_player.shape[0]):
            if i == 0:
                current_player["cum_points"].iloc[i] = current_player["total_points"].iloc[i]
                current_player["local_val"].iloc[i] = current_player["total_points"].iloc[i] / current_player["cost"].iloc[i]
                current_player["cum_val"].iloc[i] = current_player["cum_points"].iloc[i] / current_player["cost"].iloc[i]
            else:
                current_player["cum_points"].iloc[i] = current_player["total_points"].iloc[i] + current_player["cum_points"].iloc[i-1]
                current_player["local_val"].iloc[i] = current_player["total_points"].iloc[i] / current_player["cost"].iloc[i]
                current_player["cum_val"].iloc[i] = current_player["cum_points"].iloc[i] / current_player["cost"].iloc[i]

        # Add the player frame with populated new feature values to our cumulative dataframe
        cumulative = cumulative.append(current_player)
    
    
    
    return basic_player_info, players, cumulative

In [23]:
bpi, players, cumulative_pts = fpl_calls()

/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/an

In [33]:
# Save basic player info to csv
bpi.to_csv("basic_player_info.csv", index=False)

# Save the player's season data to csv
players.to_csv("players.csv", index=False)

# Save the cumulative points and vale set to csv
cumulative_pts.to_csv("cumulative_points.csv", index=False)